In [1]:
import  torch, os
import  numpy as np
import  scipy.stats
from    torch.utils.data import DataLoader
from    torch.optim import lr_scheduler
import  random, sys, pickle
from utils.results.dataloaders_2 import train_data_gen , test_data_gen
# from maml_meta import Meta
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from    torch import optim
from maml_learner import Learner
from    torch.nn import functional as F
from    copy import deepcopy

import json
import shutil
import torchvision.utils as vutils

In [2]:
with open("results_configs/evaluation_5shot2distractor15_with_data_augmentation_test_guasian_noise_certain.json") as json_file:
    args = json.load(json_file)
print(args)
writer = SummaryWriter("results_runs/" + args["save_path"])

{'epoch': 60000, 'n_way': 5, 'k_spt': 5, 'k_qry': 10, 'data_augmentation_num': 3, 'img_sz': 84, 'task_num': 5, 'img_c': 3, 'meta_lr': 0.001, 'update_lr': 0.01, 'update_step': 5, 'update_step_test': 10, 'loss': 'cross_entropy', 'min_learning_rate': 1e-15, 'number_of_training_steps_per_iter': 5, 'multi_step_loss_num_epochs': 10, 'spy_gan_num': 1, 'qry_gan_num': 5, 'num_distractor': 2, 'gan': 0, 'spy_distractor_num': 1, 'qry_distractor_num': 10, 'batch_for_gradient': 25, 'fm': 32, 'no_save': 0, 'learn_inner_lr': 0, 'create_graph': 0, 'msl': 0, 'single_fast_test': 0, 'consine_schedule': 12000, 'eta_min': 1e-06, 'save_path': 'evaluation_5shot2distractor15_with_data_augmentation_test_guasian_noise_certain'}


In [3]:
# BASE
fm = args["fm"]
config = [
    ("conv2d", [fm, 3, 3, 3, 1, 0]),
    ("leakyrelu", [0.2,True]),
    ("bn", [fm]),
    ("max_pool2d", [2, 2, 0]),
    ("conv2d", [fm, fm, 3, 3, 1, 0]),
    ("leakyrelu", [0.2,True]),
    ("bn", [fm]),
    ("max_pool2d", [2, 2, 0]),
    ("conv2d", [fm, fm, 3, 3, 1, 0]),
    ("leakyrelu", [0.2,True]),
    ("bn", [fm]),
    ("max_pool2d", [2, 2, 0]),
    ("conv2d", [fm, fm, 3, 3, 1, 0]),
    ("leakyrelu", [0.2,True]),
    ("bn", [fm]),
    ("max_pool2d", [2, 1, 0]),
    ("flatten", []),
    ("linear", [6, fm * 5 * 5])
]

In [4]:
train_data_generator = train_data_gen(args)
test_data_generator = test_data_gen(args)

load datasets/BelgiumTSC
load complete time 0.4136190414428711
load datasets/ArTS
load complete time 0.40228891372680664
load datasets/chinese_traffic_sign
load complete time 0.7370374202728271
load datasets/CVL
load complete time 0.5700554847717285
load datasets/FullJCNN2013
load complete time 0.2710561752319336
load datasets/logo_2k
load complete time 1.0366477966308594
load datasets/GTSRB
load complete time 0.10033535957336426
load datasets/DFG
load complete time 0.034989118576049805


In [5]:
class Meta(nn.Module):
    """
    Meta Learner
    """
    def __init__(self, args, config):
        """

        :param args:
        """
        super(Meta, self).__init__()

        self.update_lr = args["update_lr"]
        self.meta_lr = args["meta_lr"]
        self.n_way = args["n_way"]
        self.k_spt = args["k_spt"]
        self.k_qry = args["k_qry"]
        self.task_num = args["task_num"]
        self.update_step = args["update_step"]
        self.update_step_test = args["update_step_test"]
        self.distractor = args["num_distractor"]
        self.net = Learner(config, args["img_c"], args["img_sz"])
        self.meta_optim = optim.Adam(self.net.parameters(), lr=self.meta_lr)
        if args["consine_schedule"]:
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.meta_optim, T_max=args["consine_schedule"], eta_min=args["eta_min"])
        self.device = torch.device("cuda")
        self.gan = args["gan"]
        self.multi_step_loss_num_epochs = args["multi_step_loss_num_epochs"]
        
    def get_per_step_loss_importance_vector(self):

        loss_weights = np.ones(shape=(self.update_step)) * (
                1.0 / self.update_step)
        decay_rate = 1.0 / self.update_step / self.multi_step_loss_num_epochs
        min_value_for_non_final_losses = 0.03 / self.update_step
        for i in range(len(loss_weights) - 1):
            curr_value = np.maximum(loss_weights[i] - (self.current_epoch * decay_rate), min_value_for_non_final_losses)
            loss_weights[i] = curr_value

        curr_value = np.minimum(
            loss_weights[-1] + (self.current_epoch * (self.update_step - 1) * decay_rate),
            1.0 - ((self.update_step - 1) * min_value_for_non_final_losses))
        loss_weights[-1] = curr_value
        loss_weights = torch.Tensor(loss_weights).to(device=self.device)
        return loss_weights

    def forward(self, x_spt, y_spt, x_qry, y_qry, current_epoch,unlabel_spt_image=None, unlabel_qry_image=None,gan_spt=None, gan_qry=None):
        """

        :param x_spt:   [b, setsz, c_, h, w]
        :param y_spt:   [b, setsz]
        :param x_qry:   [b, querysz, c_, h, w]
        :param y_qry:   [b, querysz]
        :return:
        """
        self.current_epoch = current_epoch
        per_step_loss_importance_vectors = self.get_per_step_loss_importance_vector()
        task_num, setsz, c_, h, w = x_spt.size()
        querysz = x_qry.size(1)
        if self.gan :
            gan_sptsz = gan_spt.size(1)
            gan_qrysz = gan_qry.size(1)
        else:
            gan_sptsz = 0
            gan_qrysz = 0
        if self.distractor or self.gan:
            corrects = {}
            corrects["total_query_nway"] = np.zeros(self.update_step + 1)
            if self.distractor:
                unlabel_querysz = unlabel_qry.size(1)
                corrects["query_nway_recall"] = np.zeros(self.update_step + 1)
                corrects["label_query_nway_recall"] = np.zeros(self.update_step + 1)
                corrects["distractor_query_nway_recall"] = np.zeros(self.update_step + 1)
            if self.gan :
                corrects["gan_query_nway"] = np.zeros(self.update_step + 1)
        else:
            corrects = {key: np.zeros(self.update_step + 1) for key in 
                [
                "query_nway_recall",
                "label_query_nway_recall"
                ]}
        losses_q = [0 for _ in range(self.update_step + 1)]  # losses_q[i] is the loss on step i

        for i in range(task_num):
            spt_image = x_spt[i]
            spt_label = y_spt[i]
            qry_image = x_qry[i]
            qry_label = y_qry[i]
            if self.distractor:
                spt_image = torch.concat((spt_image,unlabel_spt[i]))
                spt_unlabel_label = torch.full((unlabel_spt.size(1),), 5, dtype=torch.long,device=self.device)
                spt_label = torch.cat((spt_label,spt_unlabel_label))
                qry_image = torch.concat((qry_image,unlabel_qry[i]))
                qry_unlabel_label = torch.full((unlabel_qry.size(1),), 5, dtype=torch.long,device=self.device)
                qry_label = torch.cat((qry_label,qry_unlabel_label))
            if self.gan :
                spt_image = torch.concat((spt_image,gan_spt[i]))
                spt_gan_label = torch.full((gan_spt.size(1),), 5, dtype=torch.long,device=self.device)
                spt_label = torch.cat((spt_label,spt_gan_label))
                qry_image = torch.concat((qry_image,gan_qry[i]))
                qry_gan_label = torch.full((gan_qry.size(1),), 5, dtype=torch.long,device=self.device)
                qry_label = torch.cat((qry_label,qry_gan_label))

            # 1. run the i-th task and compute loss for k=0
            logits = self.net(spt_image, vars=None, bn_training=True)
            loss = F.cross_entropy(logits, spt_label)
            grad = torch.autograd.grad(loss, self.net.parameters())
            fast_weights = list(map(lambda p: p[1] - self.update_lr * p[0], zip(grad, self.net.parameters())))

            # this is the loss and accuracy before first update
            with torch.no_grad():
                # [setsz, nway]
                if self.distractor or self.gan:
                    total_logits_q = self.net(qry_image, self.net.parameters(), bn_training=False)
                    total_pred_q = F.softmax(total_logits_q, dim=1).argmax(dim=1)
                    total_q_correct = torch.eq(total_pred_q, qry_label).sum().item()
                    corrects["total_query_nway"][0] += total_q_correct
                    loss_q = F.cross_entropy(total_logits_q, qry_label)
                    losses_q[0] += loss_q
                    if self.distractor:
                        label_logits_q = self.net(x_qry[i], self.net.parameters(), bn_training=False)
                        label_pred_q = F.softmax(label_logits_q, dim=1).argmax(dim=1)
                        label_pred_q_correct = torch.eq(label_pred_q, y_qry[i]).sum().item()
                        corrects["label_query_nway_recall"][0] += label_pred_q_correct
                        label_pred_q = F.softmax(label_logits_q[:,:-1], dim=1).argmax(dim=1)
                        label_pred_q_correct = torch.eq(label_pred_q, y_qry[i]).sum().item()
                        corrects["query_nway_recall"][0] += label_pred_q_correct
                        

                        unlabel_logits_q = self.net(unlabel_qry[i], self.net.parameters(), bn_training=False)
                        unlabel_pred_q = F.softmax(unlabel_logits_q, dim=1).argmax(dim=1)
                        other = torch.eq(unlabel_pred_q, qry_unlabel_label).sum().item()
                        corrects["distractor_query_nway_recall"][0] += other
                    if self.gan :
                        gan_logits_q = self.net(gan_qry[i], self.net.parameters(), bn_training=False)
                        gan_pred_q = F.softmax(gan_logits_q, dim=1).argmax(dim=1)
                        gan_counts = torch.eq(gan_pred_q, qry_gan_label).sum().item()
                        corrects["gan_query_nway"][0] += gan_counts
                else:
                    logits_q = self.net(qry_image, self.net.parameters(), bn_training=True)
                    loss_q = F.cross_entropy(logits_q, qry_label)
                    pred_q = F.softmax(logits_q[:,:-1], dim=1).argmax(dim=1)
                    q_discrim_correct = torch.eq(pred_q, qry_label).sum().item()
                    corrects["query_nway_recall"][0] += q_discrim_correct
                    pred_q = F.softmax(logits_q, dim=1).argmax(dim=1)
                    q_discrim_correct = torch.eq(pred_q, qry_label).sum().item()
                    corrects["label_query_nway_recall"][0] += q_discrim_correct
                    losses_q[0] += loss_q
            # this is the loss and accuracy after the first update
            with torch.no_grad():
                # [setsz, nway]
                if self.distractor or self.gan:
                    
                    total_logits_q = self.net(qry_image,fast_weights , bn_training=False)
                    total_pred_q = F.softmax(total_logits_q, dim=1).argmax(dim=1)
                    total_q_correct = torch.eq(total_pred_q, qry_label).sum().item()
                    corrects["total_query_nway"][1] += total_q_correct
                    loss_q = F.cross_entropy(total_logits_q, qry_label)
                    losses_q[1] += loss_q
                    if self.distractor:
                        label_logits_q = self.net(x_qry[i], fast_weights, bn_training=False)
                        label_pred_q = F.softmax(label_logits_q, dim=1).argmax(dim=1)
                        label_pred_q_correct = torch.eq(label_pred_q, y_qry[i]).sum().item()
                        corrects["label_query_nway_recall"][1] += label_pred_q_correct
                        label_pred_q = F.softmax(label_logits_q[:,:-1], dim=1).argmax(dim=1)
                        label_pred_q_correct = torch.eq(label_pred_q, y_qry[i]).sum().item()
                        corrects["query_nway_recall"][1] += label_pred_q_correct
                        
                        unlabel_logits_q = self.net(unlabel_qry[i], fast_weights, bn_training=False)
                        unlabel_pred_q = F.softmax(unlabel_logits_q, dim=1).argmax(dim=1)
                        other = torch.eq(unlabel_pred_q, qry_unlabel_label).sum().item()
                        corrects["distractor_query_nway_recall"][1] += other
                    if self.gan :
                        gan_logits_q = self.net(gan_qry[i], fast_weights, bn_training=False)
                        gan_pred_q = F.softmax(gan_logits_q, dim=1).argmax(dim=1)
                        gan_counts = torch.eq(gan_pred_q, qry_gan_label).sum().item()
                        corrects["gan_query_nway"][1] += gan_counts
                else:
                    logits_q = self.net(qry_image, fast_weights, bn_training=False)
                    loss_q = F.cross_entropy(logits_q, qry_label)
                    pred_q = F.softmax(logits_q, dim=1).argmax(dim=1)
                    q_discrim_correct = torch.eq(pred_q, qry_label).sum().item()
                    corrects["query_nway_recall"][1] += q_discrim_correct
                    pred_q = F.softmax(logits_q[:,:-1], dim=1).argmax(dim=1)
                    q_discrim_correct = torch.eq(pred_q, qry_label).sum().item()
                    corrects["label_query_nway_recall"][1] += q_discrim_correct
                    losses_q[1] += loss_q

            for k in range(1, self.update_step):
                # 1. run the i-th task and compute loss for k=1~K-1
                logits = self.net(spt_image, fast_weights, bn_training=True)
                loss = F.cross_entropy(logits, spt_label)
                # 2. compute grad on theta_pi
                grad = torch.autograd.grad(loss, fast_weights)
                # 3. theta_pi = theta_pi - train_lr * grad
                fast_weights = list(map(lambda p: p[1] - self.update_lr * p[0], zip(grad, fast_weights)))

                logits_q = self.net(qry_image, fast_weights, bn_training=True)
                # loss_q will be overwritten and just keep the loss_q on last update step.
                loss_q = F.cross_entropy(logits_q, qry_label)
                losses_q[k + 1] += loss_q
                with torch.no_grad():
                    if self.distractor or self.gan:
                        total_logits_q = self.net(qry_image, fast_weights, bn_training=False)
                        total_pred_q = F.softmax(total_logits_q, dim=1).argmax(dim=1)
                        total_q_correct = torch.eq(total_pred_q, qry_label).sum().item()
                        corrects["total_query_nway"][k+1] += total_q_correct
                        if self.distractor:
                            label_logits_q = self.net(x_qry[i], fast_weights, bn_training=False)
                            label_pred_q = F.softmax(label_logits_q, dim=1).argmax(dim=1)
                            label_pred_q_correct = torch.eq(label_pred_q, y_qry[i]).sum().item()
                            corrects["label_query_nway_recall"][k+1] += label_pred_q_correct
                            label_pred_q = F.softmax(label_logits_q[:,:-1], dim=1).argmax(dim=1)
                            label_pred_q_correct = torch.eq(label_pred_q, y_qry[i]).sum().item()
                            corrects["query_nway_recall"][k+1] += label_pred_q_correct
                            
                            unlabel_logits_q = self.net(unlabel_qry[i], fast_weights, bn_training=False)
                            unlabel_pred_q = F.softmax(unlabel_logits_q, dim=1).argmax(dim=1)
                            other = torch.eq(unlabel_pred_q, qry_unlabel_label).sum().item()
                            corrects["distractor_query_nway_recall"][k+1] += other
                        if self.gan :
                            gan_logits_q = self.net(gan_qry[i], fast_weights, bn_training=False)
                            gan_pred_q = F.softmax(gan_logits_q, dim=1).argmax(dim=1)
                            gan_counts = torch.eq(gan_pred_q, qry_gan_label).sum().item()
                            corrects["gan_query_nway"][k+1] += gan_counts
                    else:
                        pred_q = F.softmax(logits_q, dim=1).argmax(dim=1)
                        q_discrim_correct = torch.eq(pred_q, qry_label).sum().item()
                        corrects["query_nway_recall"][k+1] += q_discrim_correct
                        pred_q = F.softmax(logits_q[:,:-1], dim=1).argmax(dim=1)
                        q_discrim_correct = torch.eq(pred_q, qry_label).sum().item()
                        corrects["label_query_nway_recall"][k+1] += q_discrim_correct
        # end of all tasks
        # sum over all losses on query set across all tasks
        loss_q = 0

        for num_step, loss in enumerate(losses_q[1:]):
            loss_q = loss_q + per_step_loss_importance_vectors[num_step] * losses_q[-1] / task_num
        # optimize theta parameters
        self.meta_optim.zero_grad()
        loss_q.backward()

        self.meta_optim.step()
        
        accs = {}

        if self.distractor:
            accs["label_query_nway_recall"] = corrects["label_query_nway_recall"] / (task_num * querysz)
            accs["query_nway_recall"] = corrects["query_nway_recall"] / (task_num * querysz)
            accs["distractor_query_nway_recall"] = corrects["distractor_query_nway_recall"] / (task_num * unlabel_querysz)

        else:
            accs["query_nway_recall"] = corrects["query_nway_recall"] / (task_num * querysz)
            accs["label_query_nway_recall"] = corrects["label_query_nway_recall"] / (task_num * querysz)
        return accs,loss_q


    def finetunning(self, x_spt, y_spt, x_qry, y_qry, unlabel_spt=None, unlabel_qry=None, gan_spt=None, gan_qry=None):

        assert len(x_spt.shape) == 4
        querysz = x_qry.size(0)

        corrects = {}
        corrects["total_query_nway"] = np.zeros(self.update_step_test + 1)

        unlabel_querysz = unlabel_qry.size(0)

        corrects["query_nway_recall"] = np.zeros(self.update_step_test + 1)
        corrects["distractor_query_nway_recall"] = np.zeros(self.update_step_test + 1)

        # in order to not ruin the state of running_mean/variance and bn_weight/bias
        # we finetunning on the copied model instead of self.net
        net = deepcopy(self.net)
        spt_image = x_spt
        spt_label = y_spt
        qry_image = x_qry
        qry_label = y_qry
        if self.distractor:
            spt_image = torch.concat((spt_image,unlabel_spt))
            spt_unlabel_label = torch.full((unlabel_spt.size(0),), 5, dtype=torch.long,device=self.device)
            spt_label = torch.cat((spt_label,spt_unlabel_label))

        qry_image = torch.concat((qry_image,unlabel_qry))
        qry_unlabel_label = torch.full((unlabel_qry.size(0),), 5, dtype=torch.long,device=self.device)
        qry_label = torch.cat((qry_label,qry_unlabel_label))

        # 1. run the i-th task and compute loss for k=0
        logits = net(spt_image)
        loss = F.cross_entropy(logits, spt_label)

        grad = torch.autograd.grad(loss, net.parameters())
        fast_weights = list(map(lambda p: p[1] - self.update_lr * p[0], zip(grad, net.parameters())))

        # this is the loss and accuracy before first update
        with torch.no_grad():

            total_logits_q = self.net(qry_image,self.net.parameters() , bn_training=False)
            total_pred_q = F.softmax(total_logits_q, dim=1).argmax(dim=1)
            total_q_correct = torch.eq(total_pred_q, qry_label).sum().item()
            corrects["total_query_nway"][1] += total_q_correct
            loss_q = F.cross_entropy(total_logits_q, qry_label)

            label_logits_q = self.net(x_qry, self.net.parameters(), bn_training=False)
            label_pred_q = F.softmax(label_logits_q[:,:-1], dim=1).argmax(dim=1)
            label_pred_q_correct = torch.eq(label_pred_q, y_qry).sum().item()
            corrects["query_nway_recall"][0] += label_pred_q_correct

            unlabel_logits_q = self.net(unlabel_qry, self.net.parameters(), bn_training=False)
            unlabel_pred_q = F.softmax(unlabel_logits_q, dim=1).argmax(dim=1)
            other = torch.eq(unlabel_pred_q, qry_unlabel_label).sum().item()
            corrects["distractor_query_nway_recall"][0] += other


        # this is the loss and accuracy after the first update
        with torch.no_grad():

            total_logits_q = self.net(qry_image,fast_weights , bn_training=False)
            total_pred_q = F.softmax(total_logits_q, dim=1).argmax(dim=1)
            
            total_q_correct = torch.eq(total_pred_q, qry_label).sum().item()
            corrects["total_query_nway"][1] += total_q_correct
            loss_q = F.cross_entropy(total_logits_q, qry_label)

            label_logits_q = self.net(x_qry, fast_weights, bn_training=False)
            label_pred_q = F.softmax(label_logits_q[:,:-1], dim=1).argmax(dim=1)
            label_pred_q_correct = torch.eq(label_pred_q, y_qry).sum().item()
            corrects["query_nway_recall"][1] += label_pred_q_correct

            unlabel_logits_q = self.net(unlabel_qry, fast_weights, bn_training=False)
            
            unlabel_pred_q = F.softmax(unlabel_logits_q, dim=1).argmax(dim=1)
            other = torch.eq(unlabel_pred_q, qry_unlabel_label).sum().item()
            corrects["distractor_query_nway_recall"][1] += other


        for k in range(1, self.update_step_test):
            # 1. run the i-th task and compute loss for k=1~K-1
            logits = net(spt_image, fast_weights, bn_training=True)
            loss = F.cross_entropy(logits, spt_label)
            # 2. compute grad on theta_pi
            grad = torch.autograd.grad(loss, fast_weights)
            # 3. theta_pi = theta_pi - train_lr * grad
            fast_weights = list(map(lambda p: p[1] - self.update_lr * p[0], zip(grad, fast_weights)))

            logits_q = net(qry_image, fast_weights, bn_training=True)
            # loss_q will be overwritten and just keep the loss_q on last update step.
            loss_q = F.cross_entropy(logits_q, qry_label)

            with torch.no_grad():

                total_logits_q = self.net(qry_image,fast_weights , bn_training=False)

                total_pred_q = F.softmax(total_logits_q, dim=1).argmax(dim=1)
                total_q_correct = torch.eq(total_pred_q, qry_label).sum().item()
                corrects["total_query_nway"][k+1] += total_q_correct
                loss_q = F.cross_entropy(total_logits_q, qry_label)

                label_logits_q = self.net(x_qry, fast_weights, bn_training=False)
                label_pred_q = F.softmax(label_logits_q[:,:-1], dim=1).argmax(dim=1)
                label_pred_q_correct = torch.eq(label_pred_q, y_qry).sum().item()
                corrects["query_nway_recall"][k+1] += label_pred_q_correct

                unlabel_logits_q = self.net(unlabel_qry, fast_weights, bn_training=False)
                unlabel_pred_q = F.softmax(unlabel_logits_q, dim=1).argmax(dim=1)
                other = torch.eq(unlabel_pred_q, qry_unlabel_label).sum().item()
                corrects["distractor_query_nway_recall"][k+1] += other
        del net
        accs = {}

        accs["total_query_nway"] = corrects["total_query_nway"] / (querysz + unlabel_querysz)
        accs["query_nway_recall"] = corrects["query_nway_recall"] / querysz
        accs["distractor_query_nway_recall"] = corrects["distractor_query_nway_recall"] / (unlabel_querysz)

        return accs


In [6]:
device = torch.device("cuda")
maml = Meta(args, config).to(device)

In [7]:
tmp = filter(lambda x: x.requires_grad, maml.parameters())
num = sum(map(lambda x: np.prod(x.shape), tmp))

In [8]:
import glob
path = args["save_path"]
model_path = path[11:]
model_list = glob.glob("model_results/" + model_path + "/*")
model_sort = []
str_lenth = len("model_results/" + model_path + "/model_step")
for index, num in enumerate(model_list):
    num = num[str_lenth:-3]
    model_sort.append(num)
model_sort = [int(x) for x in model_sort]
model_sort.sort()
model_sort = [str(x) for x in model_sort]
for index, num in enumerate(model_sort):
    model_list[index] = "model_results/" + model_path + "/model_step" + num + ".pt"


In [9]:
import time
for model_name in model_list:
    t0 = time.time()
    step = int(model_name[str_lenth:-3])
    model_path = model_name

    maml = Meta(args, config).to(device)
    maml.load_state_dict(torch.load(model_path))
    
    db_test = DataLoader(test_data_generator, 1, shuffle=True, num_workers=1, pin_memory=True)
    accs_all_test = {
                    "total_query_nway":[],
                    "distractor_query_nway_recall":[],
                    "query_nway_recall":[],
                    "label_query_nway_recall":[],
                    "gan_query_nway":[]
    }
    for test_data in db_test:

        x_spt, y_spt, x_qry, y_qry, unlabel_spt, unlabel_qry = test_data
        x_spt, y_spt, x_qry, y_qry, unlabel_spt, unlabel_qry = x_spt.squeeze(0).to(device), y_spt.squeeze(0).to(device), \
                                     x_qry.squeeze(0).to(device), y_qry.squeeze(0).to(device),\
                                    unlabel_spt.squeeze(0).to(device), unlabel_qry.squeeze(0).to(device)

        accs = maml.finetunning(x_spt, y_spt, x_qry, y_qry, unlabel_spt, unlabel_qry)

        if "total_query_nway" in accs:
            accs_all_test["total_query_nway"].append(accs["total_query_nway"])
        if "label_query_nway_recall" in accs:
            accs_all_test["label_query_nway_recall"].append(accs["label_query_nway_recall"])
        if "distractor_query_nway_recall" in accs:
            accs_all_test["distractor_query_nway_recall"].append(accs["distractor_query_nway_recall"])
        if "gan_query_nway" in accs:
            accs_all_test["gan_query_nway"].append(accs["gan_query_nway"])
        if "query_nway_recall" in accs:
            accs_all_test["query_nway_recall"].append(accs["query_nway_recall"])
    # [b, update_step+1]
    if "total_query_nway" in accs:
        accs["total_query_nway"] = np.array(accs_all_test["total_query_nway"]).mean(axis=0).astype(np.float16)
        writer.add_scalar("Accuracy/test_total_query_nway_accuracy", accs["total_query_nway"][-1], step)
    if "label_query_nway_recall" in accs:
        accs["label_query_nway_recall"] = np.array(accs_all_test["label_query_nway_recall"]).mean(axis=0).astype(np.float16)

        writer.add_scalar("Accuracy/test_label_query_nway_accuracy", accs["label_query_nway_recall"][-1], step)
    if "distractor_query_nway_recall" in accs:
        accs["distractor_query_nway_recall"] = np.array(accs_all_test["distractor_query_nway_recall"]).mean(axis=0).astype(np.float16)
        writer.add_scalar("Accuracy/test_distractor_query_nway_recall_accuracy", accs["distractor_query_nway_recall"][-1], step)
    if "gan_query_nway" in accs:
        accs["gan_query_nway"] = np.array(accs_all_test["gan_query_nway"]).mean(axis=0).astype(np.float16)
        writer.add_scalar("Accuracy/test_gan_query_nway_accuracy", accs["gan_query_nway"][-1], step)
    if "query_nway_recall" in accs:
        accs["query_nway_recall"] = np.array(accs_all_test["query_nway_recall"]).mean(axis=0).astype(np.float16)
        writer.add_scalar("Accuracy/test_query_nway_accuracy", accs["query_nway_recall"][-1], step)
    t1 = time.time()
    print(step, t1 - t0)
    print("Test acc:", accs)

    step += 1

0 5.785412311553955
Test acc: {'total_query_nway': array([0.    , 0.5054, 0.414 , 0.4436, 0.468 , 0.4841, 0.4954, 0.5044,
       0.51  , 0.515 , 0.521 ], dtype=float16), 'query_nway_recall': array([0.2047, 0.3967, 0.4807, 0.5127, 0.538 , 0.5557, 0.5654, 0.5747,
       0.58  , 0.5854, 0.5903], dtype=float16), 'distractor_query_nway_recall': array([0.3   , 0.1813, 0.225 , 0.2338, 0.2488, 0.2588, 0.27  , 0.2688,
       0.27  , 0.27  , 0.2788], dtype=float16)}
100 5.6333231925964355
Test acc: {'total_query_nway': array([0.    , 0.3486, 0.37  , 0.462 , 0.504 , 0.5254, 0.5396, 0.55  ,
       0.555 , 0.5625, 0.5654], dtype=float16), 'query_nway_recall': array([0.1853, 0.4714, 0.5537, 0.5845, 0.6064, 0.6147, 0.62  , 0.627 ,
       0.633 , 0.636 , 0.638 ], dtype=float16), 'distractor_query_nway_recall': array([0.9375, 0.991 , 0.8765, 0.7114, 0.6226, 0.569 , 0.5425, 0.516 ,
       0.4988, 0.4863, 0.4724], dtype=float16)}
200 5.644435882568359
Test acc: {'total_query_nway': array([0.    , 0.368 ,